# テーマ：MNISTデータセットを多層パーセプトロン（MLP）で学習する
## 制約
* 全体の実行時間は60分以内

## 今回参考にしたもの

* [Get started with TensorFlow 2.0 for experts](https://www.tensorflow.org/alpha/tutorials/quickstart/advanced)

## 0. Install TensorFlow 2.0

In [1]:
!pip install tensorflow-gpu==2.0.0-alpha0

## 1. Prepare dataset

In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf

dataset = tfds.load('mnist', as_supervised=True)
mnist_train, mnist_test = dataset['train'], dataset['test']

In [0]:
def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

In [0]:
mnist_train = mnist_train.map(convert_types).shuffle(10000).batch(32)
mnist_test = mnist_test.map(convert_types).batch(32)

## 2. Define model

In [0]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = Flatten() # (28, 28, 1) -> 784
        self.d1 = Dense(128, activation = 'relu') # 784 -> 128
        self.d2 = Dense(10, activation = 'softmax') # 128 -> 10
        
    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
model = MyModel()

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## 3. Prepare training

In [0]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')

test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')

In [0]:
@tf.function
def train_step(image, label):
    with tf.GradientTape() as tape:
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label, predictions)

In [0]:
@tf.function
def test_step(image, label):
    predictions = model(image)
    t_loss = loss_object(label, predictions)
    
    test_loss(t_loss)
    test_accuracy(label, predictions)

## 4. Train

In [10]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for image, label in mnist_train:
        train_step(image, label)
        
    for test_image, test_label in mnist_test:
        test_step(test_image, test_label)
        
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(), test_accuracy.result() * 100))

Epoch 1, Loss: 0.2557194232940674, Accuracy: 92.70999908447266, Test Loss: 0.1378147453069687, Test Accuracy: 95.83000183105469
Epoch 2, Loss: 0.18357810378074646, Accuracy: 94.70499420166016, Test Loss: 0.11774928122758865, Test Accuracy: 96.55000305175781
Epoch 3, Loss: 0.1481676548719406, Accuracy: 95.68777465820312, Test Loss: 0.10903067141771317, Test Accuracy: 96.7366714477539
Epoch 4, Loss: 0.12559226155281067, Accuracy: 96.33708953857422, Test Loss: 0.10092934966087341, Test Accuracy: 96.95500183105469
Epoch 5, Loss: 0.10931996256113052, Accuracy: 96.79499816894531, Test Loss: 0.0945318341255188, Test Accuracy: 97.14400482177734
